In [1]:
import numpy as np

In [37]:
s = np.array([0, 0, 1, 0, 2, 2, 2])
n = 3
m = 4
seq= np.array([[0, 1, 1, 1],
               [0, 0, 1, 0],
               [0, 0, 1, 0],
               [0, 1, 0, 0],
               [1, 0, 0, 1],
               [1, 1, 0, 0],
               [0, 1, 0, 1]])
pairs = [(s[t-1], s[t]) for t in range(1, len(s))]

counts = np.bincount(s)
P = np.zeros((n, m))
A = np.zeros((n, n))
C = np.zeros(n)
print(pairs)
print(counts)

C[s[0]] += 1
# counts[i] = the total number of occurrences of state i in the sequence
counts = np.bincount(s)
# pairs[i] = (s_{t}, s_{t+1}), 1<= t <= T-1, T: total length of the obsevation
pairs = [(s[t-1], s[t]) for t in range(1, len(s))]

for i in range(n):
    for j in range(n):
        A[i][j] = pairs.count((i, j)) / (counts[i] - (s[-1] == i))
    for j in range(m):
        P[i][j] = sum([(s[t] == i) * seq[t][j] for t in range(len(s))]) / counts[i]
        
print(C)
print(A)
print(P)

[(0, 0), (0, 1), (1, 0), (0, 2), (2, 2), (2, 2)]
[3 1 3]
[1. 0. 0.]
[[0.33333333 0.33333333 0.33333333]
 [1.         0.         0.        ]
 [0.         0.         1.        ]]
[[0.         0.66666667 0.66666667 0.33333333]
 [0.         0.         1.         0.        ]
 [0.66666667 0.66666667 0.         0.66666667]]


In [1]:
import analysis
import numpy as np

s = np.array([0, 0, 1, 0, 2, 2, 2])
n = 3
m = 4
seq = np.array([[0, 1, 1, 1],
               [0, 0, 1, 0],
               [0, 0, 1, 0],
               [0, 1, 0, 0],
               [1, 0, 0, 1],
               [1, 1, 0, 0],
               [0, 1, 0, 1]])
hmm = analysis.HMM4binary_sensors()
hmm.fit_with_true_hidden_states(n, m, seq, s)
hmm.predict_states(seq)

(array([0, 1, 0, 2, 2, 2, 2]), 5.873606637142921e-07)

In [49]:
a = np.arange(5)
b = np.arange(5) + 2
b[2] += 15
print(a)
print(b)
print(a * b)
print(np.max(a * b))
print(np.argmax(a * b))

[0 1 2 3 4]
[ 2  3 19  5  6]
[ 0  3 38 15 24]
38
2
